# Imports

In [1]:
orthologous_genes_folder = '/home/olga/googledrive/TabulaMicrocebus/data/orthologous-genes'
# ! ls -lha $orthologous_genes_folder

In [2]:
import os

import anndata
import scanpy as sc
import pandas as pd
import seaborn as sns


sc.settings.verbosity = 3
sc.logging.print_versions()

pd.options.display.max_rows = 200


outdir_gdrive = "/home/olga/googledrive/TabulaMicrocebus/data/cross-species"
outdir_local = "/home/olga/data_lg/data_sm_copy/tabula-microcebus/data-objects/cross-species"
outdirs = outdir_gdrive, outdir_local

-----
anndata     0.7.5
scanpy      1.7.0rc1
sinfo       0.3.1
-----
anndata             0.7.5
backcall            0.1.0
cairo               1.18.2
colorama            0.4.4
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.0
decorator           4.4.1
get_version         2.1
h5py                2.10.0
igraph              0.7.1
importlib_metadata  0.19
ipykernel           5.1.3
ipython_genutils    0.2.0
jedi                0.15.1
joblib              0.14.1
kiwisolver          1.1.0
legacy_api_wrap     0.0.0
leidenalg           0.7.0
llvmlite            0.30.0
louvain             0.6.1
matplotlib          3.1.2
more_itertools      NA
mpl_toolkits        NA
natsort             6.2.0
numba               0.46.0
numexpr             2.7.0
numpy               1.17.3
packaging           20.8
pandas              1.1.4
parso               0.5.1
pexpect             4.7.0
pickleshare         0.7.5
pkg_resources       NA
prompt_toolkit      3.0.2
ptyprocess          0.6.0
pyg

## Load mega-object

In [ ]:
%%time

h5ad = os.path.join(
    outdir_local,
    "concatenated__human-lemur-mouse_10x__one2one_orthologs__july2021.h5ad",
)
! ls -lha $h5ad
combined = sc.read(h5ad)

-rw-r--r-- 1 olga czb 7.4G Jul 15 11:12 /home/olga/data_lg/data_sm_copy/tabula-microcebus/data-objects/cross-species/concatenated__human-lemur-mouse_10x__one2one_orthologs__july2021.h5ad


In [ ]:
!ls  -lha $h5ad

In [ ]:
sc.pl.dotplot(combined, var_names=['SFTPC'], groupby='narrow_group')

In [ ]:
combined.obs.tissue.value_counts()

In [ ]:
combined.obs.groupby(['species_batch', 'tissue']).size()

# Subset to Lung, Blood, Muscle and write to file

In [ ]:
tissues = "Lung", "Blood", "Muscle", "Bladder"
tissues = {
#     "lung": ("Lung",),
#     "blood": ("Blood",),
    "muscle": ("Muscle", "Limb_Muscle", "Limb_muscle"),
#     "bladder": ("Bladder",),
}


tissue_to_celltype_col = {
#     'lung': 'narrow_group',
#     'blood': 'narrow_group',
    'muscle': 'narrow_group',
#     'bladder': 'cell_ontology_class'
}

In [ ]:
def remove_unused_obs_categories(adata):
    """Remove all unused cell ontology class, free annotations, ages, etc using pandas"""
    for col_name, col_series in adata.obs.items():
        try:
            adata.obs[col_name] = col_series.cat.remove_unused_categories()
        except AttributeError:
            continue
    return adata




for tissue_key, tissue_names in tissues.items():
    print(f'--- {tissue_key} ---')
    combined_tissue = combined[combined.obs.tissue.isin(tissue_names)]
    
    # Remove all unused cell ontology class, free annotations, ages, etc
    combined_tissue = remove_unused_obs_categories(combined_tissue)
    
    celltype_col = tissue_to_celltype_col[tissue_key]
    print(combined_tissue.obs.groupby([celltype_col, 'species_batch']).size())
    
    print(combined_tissue)
#     for d in outdirs:
    h5ad = os.path.join(
        outdir_local,
        f'concatenated__human-lemur-mouse_10x__one2one_orthologs__{tissue_key}__july2021.h5ad')

    print(h5ad)
    %time combined_tissue.write(h5ad)